In [1]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import scipy
import math
import datetime
import re
import json

In [2]:
import tensorflow as tf
from tensorflow.keras.callbacks import (ModelCheckpoint, TensorBoard, ReduceLROnPlateau,
                                        CSVLogger, EarlyStopping)
from csv import writer
from datetime import datetime

In [3]:
from load_dataset import create_dataset, read_and_random_undersampling_dataset
from callback_save_files import TrainingCallback, FitCallback
import local_settings
from anneal_simulation import AnnealSimulation
from CNN_creation import CreateModel, form_CNN_parameters

In [4]:
WORKING_DIRECTORY = local_settings.WORKING_DIRECTORY #'/home/buliabog/Diploma/'
logs_dir = f'{WORKING_DIRECTORY}Data/logs/'
file_name = '20240117 explore phase x y effect'
task_name = ''

In [5]:
logs_directory = logs_dir + file_name

In [6]:
val_filepath = f'{WORKING_DIRECTORY}Data/CODE-15/val'
train_filepath = f'{WORKING_DIRECTORY}Data/CODE-15/train'

# CNN

In [7]:
space = { 'kernel_0' : (1, 2, 4, 6),     # kernel
          'max_pooling_0': (1, 4, 8, 12, 16),      # max_pooling
          'units_0': (2, 4, 8, 12, 16, 20, 30, 50, 80, 160), # units/filters   
          'activation_0': ('swish', 'linear'), # activation
         'kernel_1' : (1, 2, 4, 6),     # kernel
          'max_pooling_1': (1, 4, 8, 12, 16),      # max_pooling
          'units_1': (2, 4, 8, 12, 16, 20, 30, 50, 80, 160), # units/filters   
          'activation_1': ('swish', 'linear'), # activation
         'kernel_2' : (1, 2, 4, 6),     # kernel
          'max_pooling_2': (1, 4, 8, 12, 16),      # max_pooling
          'units_2': (2, 4, 8, 12, 16, 20, 30, 50, 80, 160), # units/filters   
          'activation_2': ('swish', 'linear'), # activation
         'kernel_3' : (1, 2, 4, 6),     # kernel
          'max_pooling_3': (1, 4, 8, 12, 16),      # max_pooling
          'units_3': (2, 4, 8, 12, 16, 20, 30, 50, 80, 160), # units/filters   
          'activation_3': ('swish', 'linear') # activation
        }
spaces = list(space.values())

In [8]:
best_state = [1,0,9,0,2,3,5,1,1,0,9,0,3,2,7,1]
CNN_params = form_CNN_parameters(best_state, spaces)
print(json.dumps(CNN_params, indent=2))

{
  "CNN_count": 4,
  "CNN_parameters": [
    {
      "kernel": 2,
      "max_pooling": 1,
      "units": 160,
      "activation": "swish"
    },
    {
      "kernel": 4,
      "max_pooling": 12,
      "units": 20,
      "activation": "linear"
    },
    {
      "kernel": 2,
      "max_pooling": 1,
      "units": 160,
      "activation": "swish"
    },
    {
      "kernel": 6,
      "max_pooling": 8,
      "units": 50,
      "activation": "linear"
    }
  ]
}


In [9]:
def create_random_state(spaces):
    state = []
    for parameter in spaces:
        state.append(np.random.randint(len(parameter)))
    return state

In [12]:
for NN_version in range(15):
    NN_state = create_random_state(spaces)
    CNN_params = form_CNN_parameters(NN_state, spaces)
    file_number = np.random.randint(18)
    NN_state_str = f":file {file_number}|"+"|".join(np.array(NN_state, dtype=str))
    for task_name in ['ImprovedFourieMagnitudePhase'
                      , 'ImprovedFourieMagnitude'
                      , 'ImprovedFourieMagnitudeXY'
                      , 'ClassicFourieMagnitudePhase'
                      , 'ClassicFourieMagnitude'
                      , 'ClassicFourieMagnitudeXY']:
        train_dataset = read_and_random_undersampling_dataset(f'{WORKING_DIRECTORY}Data/CODE-15', 'train'
                                                              , file_num = file_number
                                                              , dataset_type = task_name)
        val_dataset   = read_and_random_undersampling_dataset(f'{WORKING_DIRECTORY}Data/CODE-15', 'val'
                                                              , file_num = file_number
                                                              , dataset_type = task_name
                                                              , undersampling = False
                                                             )

        inst_count = 0
        for inst in train_dataset:
            inst_count += 1

        lr = 0.001
        # CALLBACKS
        callbacks = [ReduceLROnPlateau(monitor='val_loss',
                                                           factor=0.1,
                                                           patience=6,
                                                           min_lr=lr / 100),
                                         EarlyStopping(patience=11,  # Patience should be larger than the one in ReduceLROnPlateau
                                                       min_delta=0.00001),
                                         FitCallback(logs_dir, file_name, val_dataset, task_name+NN_state_str)
                    ]

        if task_name[-5:] == 'Phase':
            channels = 24
        else:
            if task_name[-2:] == 'XY':
                channels = 36
            else:
                channels = 12
        try:
            model = CreateModel(CNN_params, (500,channels,), 6)
        except:
            continue
        model.summary()

        res = []        
        precision = np.zeros(101)
        recall = np.zeros(101)
        f1 = np.zeros(101)

        for run in range(3):
            model_clone = tf.keras.models.clone_model(model)

            model_clone.compile(loss=tf.keras.losses.BinaryCrossentropy() #'binary_crossentropy'
                             , optimizer=tf.keras.optimizers.Adam(lr)
                             , metrics=[tf.keras.metrics.BinaryAccuracy(name='binary_accuracy')
                                    , tf.keras.metrics.Precision(name='precision')
                                    , tf.keras.metrics.Recall(name='recall')
                                    , tf.keras.metrics.F1Score(name='f1_score', average='micro')])
            model_clone.fit( train_dataset.shuffle(train_dataset.cardinality(), reshuffle_each_iteration  = False).batch(8).repeat(70),
                            epochs=70,
                            initial_epoch=0,
                            steps_per_epoch = np.floor(inst_count/8),
                            callbacks = callbacks, 
                            validation_data=(val_dataset.batch(8)),
                            verbose=1)

            prediction = model_clone.predict(val_dataset.batch(8), verbose=0)
            try:                
                if labels.shape[0] != prediction.shape[0]:
                    labels = np.zeros(prediction.shape)
                    for instance, row in enumerate(iter(val_dataset)):
                        labels[instance, :] = row[1].numpy()  
            except UnboundLocalError:
                labels = np.zeros(prediction.shape)
                for instance, row in enumerate(iter(val_dataset)):
                    labels[instance, :] = row[1].numpy()  
            except NameError:
                labels = np.zeros(prediction.shape)
                for instance, row in enumerate(iter(val_dataset)):
                    labels[instance, :] = row[1].numpy() 

            for i in np.arange(101):
                tau = i/100.0
                true_positive =  (prediction >= tau) * labels
                true_negative = ((prediction >= tau) - 1) * (labels - 1)
                false_positive = -((prediction >= tau) * (labels - 1))
                false_negative = -((prediction >= tau) - 1) * labels
                precision[i] = np.nan_to_num(np.sum(true_positive) / (np.sum(true_positive) + np.sum(false_positive)))
                recall[i] = np.nan_to_num(np.sum(true_positive) / (np.sum(true_positive) + np.sum(false_negative)))
                f1[i] = np.nan_to_num(2*precision[i]*recall[i]/(precision[i] + recall[i]))
            i_best = np.argmax(f1)
            print(f'Task {task_name+NN_state_str}, run {run}: F1 = {f1[i_best]} on tau = {i_best}%')


Shapes: (2964, 500, 24) and (2964, 6)
Shapes: (4019, 500, 24) and (4019, 6)
<_ZipDataset element_spec=(TensorSpec(shape=(500, 24), dtype=tf.float64, name=None), TensorSpec(shape=(6,), dtype=tf.float64, name=None))>
Model: "model_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 CNN_input_X (InputLayer)    [(None, 500, 24)]         0         
                                                                 
 CNN_1 (Conv1D)              (None, 500, 8)            200       
                                                                 
 CNN_2 (Conv1D)              (None, 497, 30)           990       
                                                                 
 MaxPool_2 (MaxPooling1D)    (None, 124, 30)           0         
                                                                 
 CNN_3 (Conv1D)              (None, 123, 12)           732       
                                         

Epoch 23/70
370/370 [==============================] - 3s 9ms/step - loss: 0.1882 - binary_accuracy: 0.9326 - precision: 0.7827 - recall: 0.3623 - f1_score: 0.4707 - val_loss: 0.1151 - val_binary_accuracy: 0.9690 - val_precision: 0.3080 - val_recall: 0.3068 - val_f1_score: 0.1412 - lr: 1.0000e-04
Epoch 24/70
370/370 [==============================] - 3s 9ms/step - loss: 0.1859 - binary_accuracy: 0.9343 - precision: 0.7995 - recall: 0.3776 - f1_score: 0.4793 - val_loss: 0.1146 - val_binary_accuracy: 0.9687 - val_precision: 0.3033 - val_recall: 0.3050 - val_f1_score: 0.1404 - lr: 1.0000e-04
Epoch 25/70
370/370 [==============================] - 3s 9ms/step - loss: 0.1845 - binary_accuracy: 0.9346 - precision: 0.7984 - recall: 0.3810 - f1_score: 0.4797 - val_loss: 0.1145 - val_binary_accuracy: 0.9686 - val_precision: 0.3024 - val_recall: 0.3068 - val_f1_score: 0.1417 - lr: 1.0000e-04
Epoch 26/70
370/370 [==============================] - 3s 9ms/step - loss: 0.1841 - binary_accuracy: 0.934

370/370 [==============================] - 3s 9ms/step - loss: 0.1799 - binary_accuracy: 0.9374 - precision: 0.8217 - recall: 0.4012 - f1_score: 0.4808 - val_loss: 0.1142 - val_binary_accuracy: 0.9681 - val_precision: 0.2922 - val_recall: 0.2976 - val_f1_score: 0.1439 - lr: 1.0000e-05
Epoch 51/70
370/370 [==============================] - 3s 9ms/step - loss: 0.1799 - binary_accuracy: 0.9374 - precision: 0.8222 - recall: 0.4020 - f1_score: 0.4819 - val_loss: 0.1142 - val_binary_accuracy: 0.9681 - val_precision: 0.2922 - val_recall: 0.2976 - val_f1_score: 0.1439 - lr: 1.0000e-05
Epoch 52/70
370/370 [==============================] - 3s 9ms/step - loss: 0.1798 - binary_accuracy: 0.9373 - precision: 0.8215 - recall: 0.4006 - f1_score: 0.4816 - val_loss: 0.1142 - val_binary_accuracy: 0.9681 - val_precision: 0.2922 - val_recall: 0.2976 - val_f1_score: 0.1439 - lr: 1.0000e-05
Epoch 53/70
370/370 [==============================] - 4s 10ms/step - loss: 0.1797 - binary_accuracy: 0.9374 - precisi

C:\Users\hp\AppData\Local\Temp\ipykernel_30028\1230178777.py:92: RuntimeWarning: invalid value encountered in scalar divide
  precision[i] = np.nan_to_num(np.sum(true_positive) / (np.sum(true_positive) + np.sum(false_positive)))
C:\Users\hp\AppData\Local\Temp\ipykernel_30028\1230178777.py:94: RuntimeWarning: invalid value encountered in scalar divide
  f1[i] = np.nan_to_num(2*precision[i]*recall[i]/(precision[i] + recall[i]))


370/370 [==============================] - 5s 9ms/step - loss: 0.4898 - binary_accuracy: 0.8479 - precision: 0.1232 - recall: 0.1084 - f1_score: 0.1457 - val_loss: 0.2013 - val_binary_accuracy: 0.9685 - val_precision: 0.0868 - val_recall: 0.0425 - val_f1_score: 0.0478 - lr: 0.0010
Epoch 2/70
370/370 [==============================] - 3s 9ms/step - loss: 0.3107 - binary_accuracy: 0.9068 - precision: 0.3729 - recall: 0.0271 - f1_score: 0.1957 - val_loss: 0.1607 - val_binary_accuracy: 0.9756 - val_precision: 0.0556 - val_recall: 0.0055 - val_f1_score: 0.0654 - lr: 0.0010
Epoch 3/70
370/370 [==============================] - 4s 10ms/step - loss: 0.2863 - binary_accuracy: 0.9089 - precision: 0.5278 - recall: 0.0235 - f1_score: 0.2426 - val_loss: 0.1485 - val_binary_accuracy: 0.9759 - val_precision: 0.1579 - val_recall: 0.0166 - val_f1_score: 0.0842 - lr: 0.0010
Epoch 4/70
370/370 [==============================] - 4s 10ms/step - loss: 0.2723 - binary_accuracy: 0.9084 - precision: 0.4758 - r

...

Epoch 3/70
382/382 [==============================] - 12s 31ms/step - loss: 0.2229 - binary_accuracy: 0.9232 - precision: 0.7299 - recall: 0.2721 - f1_score: 0.4070 - val_loss: 0.8761 - val_binary_accuracy: 0.9317 - val_precision: 0.0744 - val_recall: 0.2028 - val_f1_score: 0.0593 - lr: 0.0010
Epoch 4/70
382/382 [==============================] - 12s 32ms/step - loss: 0.2089 - binary_accuracy: 0.9255 - precision: 0.7298 - recall: 0.3091 - f1_score: 0.4346 - val_loss: 0.8703 - val_binary_accuracy: 0.9383 - val_precision: 0.0653 - val_recall: 0.1501 - val_f1_score: 0.0583 - lr: 0.0010
Epoch 5/70
382/382 [==============================] - 11s 28ms/step - loss: 0.1946 - binary_accuracy: 0.9301 - precision: 0.7584 - recall: 0.3604 - f1_score: 0.4512 - val_loss: 0.8626 - val_binary_accuracy: 0.9352 - val_precision: 0.0575 - val_recall: 0.1400 - val_f1_score: 0.0558 - lr: 0.0010
Epoch 6/70
382/382 [==============================] - 9s 23ms/step - loss: 0.1875 - binary_accuracy: 0.9330 - preci

382/382 [==============================] - 8s 21ms/step - loss: 0.1315 - binary_accuracy: 0.9520 - precision: 0.8835 - recall: 0.5546 - f1_score: 0.5273 - val_loss: 4.7927 - val_binary_accuracy: 0.8061 - val_precision: 0.0286 - val_recall: 0.2556 - val_f1_score: 0.0549 - lr: 1.0000e-04
Epoch 12/70
500/500 [==============================] - 2s 5ms/step
Run started at 20240118 063606, i_best = 4, accuracy = 0.7407222083229089, f1 = 0.05530233971437253, tau_best_separated = [11 37 44  0  0 22]
Task ClassicFourieMagnitude:file 6|1|0|8|1|2|2|6|0|3|1|9|1|3|0|1|1, run 0: F1 = 0.05530233971437253 on tau = 4%
Epoch 1/70
382/382 [==============================] - 9s 20ms/step - loss: 0.3421 - binary_accuracy: 0.9010 - precision: 0.4170 - recall: 0.1794 - f1_score: 0.2900 - val_loss: 1.4729 - val_binary_accuracy: 0.8648 - val_precision: 0.0232 - val_recall: 0.1359 - val_f1_score: 0.0447 - lr: 0.0010
Epoch 2/70
382/382 [==============================] - 7s 19ms/step - loss: 0.2330 - binary_accurac

Epoch 12/70
500/500 [==============================] - 2s 4ms/step
Run started at 20240118 063939, i_best = 43, accuracy = 0.8334167292135768, f1 = 0.057560745458834627, tau_best_separated = [20 43  1  0  0 62]
Task ClassicFourieMagnitude:file 6|1|0|8|1|2|2|6|0|3|1|9|1|3|0|1|1, run 2: F1 = 0.057560745458834627 on tau = 43%
Shapes: (3062, 500, 36) and (3062, 6)
Shapes: (3997, 500, 36) and (3997, 6)
<_ZipDataset element_spec=(TensorSpec(shape=(500, 36), dtype=tf.float64, name=None), TensorSpec(shape=(6,), dtype=tf.float64, name=None))>
Model: "model_83"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 CNN_input_X (InputLayer)    [(None, 500, 36)]         0         
                                                                 
 CNN_1 (Conv1D)              (None, 499, 80)           5840      
                                                                 
 CNN_2 (Conv1D)              (None, 496, 30)  

382/382 [==============================] - 12s 31ms/step - loss: 0.1862 - binary_accuracy: 0.9340 - precision: 0.7822 - recall: 0.3980 - f1_score: 0.4644 - val_loss: 2.0390 - val_binary_accuracy: 0.7782 - val_precision: 0.0325 - val_recall: 0.3408 - val_f1_score: 0.0577 - lr: 0.0010
Epoch 7/70
382/382 [==============================] - 12s 32ms/step - loss: 0.1781 - binary_accuracy: 0.9371 - precision: 0.7923 - recall: 0.4356 - f1_score: 0.4789 - val_loss: 1.7721 - val_binary_accuracy: 0.8117 - val_precision: 0.0299 - val_recall: 0.2596 - val_f1_score: 0.0500 - lr: 0.0010
Epoch 8/70
382/382 [==============================] - 12s 32ms/step - loss: 0.1592 - binary_accuracy: 0.9433 - precision: 0.8397 - recall: 0.4788 - f1_score: 0.5019 - val_loss: 2.0151 - val_binary_accuracy: 0.8354 - val_precision: 0.0317 - val_recall: 0.2373 - val_f1_score: 0.0563 - lr: 1.0000e-04
Epoch 9/70
382/382 [==============================] - 12s 31ms/step - loss: 0.1462 - binary_accuracy: 0.9480 - precision: 

Shapes: (4010, 500, 24) and (4010, 6)
<_ZipDataset element_spec=(TensorSpec(shape=(500, 24), dtype=tf.float64, name=None), TensorSpec(shape=(6,), dtype=tf.float64, name=None))>
Shapes: (3078, 500, 12) and (3078, 6)
Shapes: (4010, 500, 12) and (4010, 6)
<_ZipDataset element_spec=(TensorSpec(shape=(500, 12), dtype=tf.float64, name=None), TensorSpec(shape=(6,), dtype=tf.float64, name=None))>
Shapes: (3078, 500, 36) and (3078, 6)
Shapes: (4010, 500, 36) and (4010, 6)
<_ZipDataset element_spec=(TensorSpec(shape=(500, 36), dtype=tf.float64, name=None), TensorSpec(shape=(6,), dtype=tf.float64, name=None))>
